# Práce s kamerou Basler v Jupyter Notebooku
<img src='images/basler-logo.jpg' style="width:50%">

Vzhledem k tomu, že software Pylon je nejen vlastní program pro připojení a manipulaci s kamerami, ale zároveň SDK, budeme používat přímo SDK. SDK je psáno v C, C++ nebo C#. Basler k tomu však v roce 2018 vytvořil wrapper do Pythonu s názvem `pypylon`.

My jsme k tomu vytvořili rozhraní s názvem `pypylon_opencv_viewer`. To umožňuje se připojit ke konkrétní kameře, zobrazit si snímky z kamery a dále nastavovat některé parametry kamery.

## Nastavení
Import potřebných knihoven

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import yaml
from pypylon import pylon 

from IPython.display import Image, display, clear_output

%matplotlib inline

Úvodni nastavení parametrů kamery. Je potřeba dopsat správné **sériové číslo** vlastní kamery. 

V případě, že se nastavuje cesta k ukládání fotek, je třeba nahradit veškerá `\` za `\\`. Při defaultním nastavení se fotky ukládají do složky s tímto ipynb.

In [ ]:
serial_number =  ### vlastní sériové číslo kamery
width = 1280
height = 1080
image_folder = '.' ### defaultní nastavení složky pro ukládání snímků

Připojení kamery pomocí `pypylonu`. Je zapotřebí zkontrolovat sériové číslo kamery.

In [ ]:
# Pypylon gets camera by serial number
info = None
for i in pylon.TlFactory.GetInstance().EnumerateDevices():
    if i.GetSerialNumber() == serial_number:
        info = i
        break
else:
    print('Camera with {} serial number not found'.format(serial_number))

# VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
if info is not None:
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
    camera.Open()

Vytvoření GUI widgetů pro manipulaci s parametry kamery. Zásady pro tvorbu JSON notace widgetů vizte na [link](https://github.com/mbalatsko/pypylon-opencv-viewer#list-of-features).

In [ ]:
# Configuration PyPylon Viewer to load features for RGB Matrix Camera
VIEWER_CONFIG_RGB_MATRIX = {
    "features": [
        {
            "name": "GainRaw",
            "type": "int",
            "step": 1,
        },
        {
            "name": "Height",
            "type": "int",
            "value": 900,
            "unit": "px",
            "step": 2,
        },
        {
            "name": "Width",
            "type": "int",
            "value": 1600,
            "unit": "px",
            "step": 2,
        },
        {
            "name": "CenterX",
            "type": "bool",
        },
        {
            "name": "CenterY",
            "type": "bool",

        },
        {
            "name": "OffsetX",
            "type": "int",
            "dependency": {"CenterX": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "OffsetY",
            "type": "int",
            "dependency": {"CenterY": False},
            "unit": "px",
            "step": 2,
        },
        {
            "name": "AcquisitionFrameRateAbs",
            "type": "int",
            "unit": "fps",
            "dependency": {"AcquisitionFrameRateEnable": True},
            "value": 30,
            "max": 150,
            "min": 1,
        },
        {
            "name": "AcquisitionFrameRateEnable",
            "type": "bool",
        },
        {
            "name": "ExposureAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
        {
            "name": "ExposureTimeAbs",
            "type": "int",
            "dependency": {"ExposureAuto": "Off"},
            "unit": "μs",
            "step": 100,
            "max": 35000,
            "min": 500,
        },
        {
            "name": "BalanceWhiteAuto",
            "type": "choice_text",
            "options": ["Off", "Once", "Continuous"],
            "style": {"button_width": "90px"}
        },
    ],
    "features_layout": [
        ("Height", "Width"), 
        ("OffsetX", "CenterX"), 
        ("OffsetY", "CenterY"), 
        ("ExposureAuto", "ExposureTimeAbs"),
        ("AcquisitionFrameRateAbs", "AcquisitionFrameRateEnable"),
        ("BalanceWhiteAuto", "GainRaw")
    ],
    "actions_layout": [
        ("StatusLabel"),
        ("SaveConfig", "LoadConfig", "ContinuousShot", "SingleShot"), 
        ("UserSet")
    ],
    "default_user_set": "UserSet3",
}

Funkce pro image processing. Zatím pouze zobrazuje obrázky.

In [ ]:
def impro(img):
    return img

## Main
Výkonný kus kódu využívající vlastní knihovnu pro práci s kamerou a knihovnou OpenCV. Umožňuje nastavení parametrů pomocí GUI widgetů.
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Continuous shot` (pro získání streamu) nebo `Single shot` (pro jeden snímek) 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
viewer = BaslerOpenCVViewer(camera)
viewer.set_configuration(VIEWER_CONFIG_RGB_MATRIX)
viewer.set_impro_function(impro)
viewer.show_interactive_panel(window_size=(width, height), image_folder=image_folder)